In [1]:
#These are the libraries you can use.  You may add any libraries directy related to threading if this is a direction
#you wish to go (this is not from the course, so it's entirely on you if you wish to use threading).  Any
#further libraries you wish to use you must email me, james@uwaterloo.ca, for permission.

from IPython.display import display, Math, Latex

import pandas as pd
import numpy as np
import numpy_financial as npf
import yfinance as yf
import matplotlib.pyplot as plt
import random
from datetime import datetime

## Group Assignment
### Team Number: 05
### Team Member Names: James Begin, Jonathan Wen, Emily Qi
### Team Strategy Chosen: Market Meet

Disclose any use of AI for this assignment below (detail where and how you used it).  Please see the course outline for acceptable uses of AI.


## Contribution Declaration

The following team members made a meaningful contribution to this assignment:

Insert Names Here.

# Introduction 
The goal of the market "meet" portfolio is to match the returns of the average of the TSX60 and S&P500 indices as closely as possibly. To do this, we look at various indicators that can provide insight into a stock's ability to match market returns. Below are the factors we plan to take into account while constructing our portfolio:  
- Volume
- Price
- Beta
- Market Cap
- Earnings and Dividend Dates
- Fund/Institutional Holdings  

## Filtering
First, we ignore any delisted stocks, or non-US and Canadian Stocks, then filter by average monthly volume:
  



In [14]:
tickers = pd.read_csv('Tickers_Example.csv')
tickerlist = [tickers.columns[0]]
tickerlist += [i for i in tickers.iloc[:,0]]
startdate = '2023-10-01'
enddate = '2024-09-30'
data1 = yf.download(tickerlist, start=startdate, end=enddate)
delisted = list(yf.shared._ERRORS.keys())
for i in delisted:
    tickerlist.remove(i)
    
exchangelist = ['NMS', 'NYQ', 'TOR']
minprice = 5
minvolume = 100000

for ticker in tickerlist:
    t = yf.Ticker(ticker)
    hist = t.history(start=startdate, end=enddate, period='1mo')
    if (t.get_history_metadata())['exchangeName'] not in exchangelist or hist['Volume'].mean() < minvolume or hist['Close'].mean() < minprice:
        tickerlist.remove(ticker)

        
    

[*********************100%***********************]  41 of 41 completed

4 Failed downloads:
['CELG', 'AGN', 'RTN', 'MON']: YFTzMissingError('$%ticker%: possibly delisted; no timezone found')


yfinance.Ticker object <AAPL>
yfinance.Ticker object <ABBV>
yfinance.Ticker object <ABT>
yfinance.Ticker object <ACN>
yfinance.Ticker object <AIG>
yfinance.Ticker object <AMZN>
yfinance.Ticker object <AXP>
yfinance.Ticker object <BA>
yfinance.Ticker object <BAC>
yfinance.Ticker object <BB.TO>
yfinance.Ticker object <BK>
yfinance.Ticker object <BLK>
yfinance.Ticker object <BMY>
yfinance.Ticker object <C>
yfinance.Ticker object <CAT>
yfinance.Ticker object <CL>
yfinance.Ticker object <KO>
yfinance.Ticker object <LLY>
yfinance.Ticker object <LMT>
yfinance.Ticker object <MO>
yfinance.Ticker object <MRK>
yfinance.Ticker object <PEP>
yfinance.Ticker object <PFE>
yfinance.Ticker object <PG>
yfinance.Ticker object <PM>
yfinance.Ticker object <PYPL>
yfinance.Ticker object <QCOM>
yfinance.Ticker object <RY.TO>
yfinance.Ticker object <SHOP.TO>
yfinance.Ticker object <T.TO>
yfinance.Ticker object <TD.TO>
yfinance.Ticker object <TXN>
yfinance.Ticker object <UNH>
yfinance.Ticker object <UNP>
yfinanc

## Beta
Beta is a measure of volatility of a security or portfolio compared to the market. A beta of 1.0 indicates that a stock follows the market's returns exactly. In this case, yfinance provides a beta value comparing each stock the S&P500 index. However, as we want to match the average of both the TSX60 and the S&P500, we will calculate beta when compared to the TSX60. Then, we can take the average of the two betas in order to accurately represent the volatility compared to the index used in this assignment.  
  
We can calculate beta using the formula (Covariance(Individual Returns, Market Returns)) / (Variance(Market Returns))